# Get Started con PySpark

### Preparando el entorno

Vamos a importar los paquetes de python que son necesarios y abrir una sesión de Spark. Puedes nombrarla como quieras, elijo 'PySpark01'

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, row_number
from pyspark.sql.window import Window

# Crear sesión de Spark
spark = SparkSession.builder.appName("PySparkShell").getOrCreate()

### Primeros pasos

Estas son algunas transformaciones rápidas que PySpark te permite aplicar sobre tus datos 

In [ ]:
# Leer archivo CSV
df = spark.read.option("header", True).option("inferSchema", True).csv("../data/movies.csv")

df.printSchema()
df.show(5)

# Conteo total de registros
print("Total de películas:", df.count())

# Filtrar y seleccionar columnas
df_filtered = df.select("title", "genre", "rating").filter(col("rating") > 8.5)

# Agrupación por género
df_grouped = df_filtered.groupBy("genre").agg(avg("rating").alias("avg_rating"))
df_grouped.orderBy(col("avg_rating").desc()).show()

### Particiones

Usando Spark, podemos particionar los datos por una o más columnas. Cuando particionamos un conjunto de datos, lo estamos dividiendo en varios archivos. Por lo tanto, podemos leer solo una partición relevante cuando nos interese, y no todos los datos.

Un archivo Parquet es un formato de archivo columnar utilizado para almacenar datos de una manera eficiente y optimizada. En lugar de guardar los datos en filas (como un archivo CSV o una tabla tradicional), Parquet los organiza por columnas, lo que permite consultas y análisis más rápidos, especialmente cuando se manejan grandes volúmenes de datos.

Un buen ejemplo útil de dividir nuestros datos sería por la columna 'año', o las columnas 'año' y 'mes', si corresponde. De esta manera, podríamos acceder a un conjunto de datos correspondientes a una fecha específica.

In [ ]:

# Aplicar función de ventana: Top 3 películas por género según mejor valoración
window_spec = Window.partitionBy("genre").orderBy(col("rating").desc())
df_with_rank = df_filtered.withColumn("rank", row_number().over(window_spec))

df_top3 = df_with_rank.filter(col("rank") <= 3)
df_top3.show()

# Guardar resultado en formato Parquet
df_top3.write.mode("overwrite").parquet("data/top_movies_by_genre.parquet")

### Detener la sesión

Finalmente, como las mejores prácticas nos enseñan, debemos detener la sesión de Spark. Eso significa que liberamos todos los recursos asignados a nuestra sesión de Spark (memoria, conexiones, pojects, etc.)

In [ ]:
# Detener la sesión de Spark
spark.stop()